In [251]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve



In [231]:
df =  pd.read_csv('../2017_07_07/TADPOLE_D1_D2.csv',low_memory=False)

In [232]:
df.head(0)


,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,DXCHANGE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,CDRSB,ADAS11,ADAS13,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,FAQ_bl,FLDSTRENG_bl,FSVERSION_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp,EXAMDATE_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,VERSION_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,FLDSTRENG_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,LONISID_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,LONIUID_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,IMAGEUID_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,RUNDATE_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,STATUS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP1_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP2_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP3_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP4_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP5_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP6_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP7_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BASETP8_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,OVERALLQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,TEMPQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,FRONTQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,PARQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,INSULAQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,OCCQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,BGQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,CWMQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,VENTQC_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST100SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST101SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST102TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST103TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST104TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST105TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST106TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST107TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST108TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST109TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST10CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,ST110TS_UCSFFSL_02_01

In [233]:
columns = df.shape[1]
print(type(columns))

<class 'int'>


In [234]:
df_sample = df.sample(frac=0.1, random_state=1)
pd.set_option('display.max_columns', None)
name_columns=df.head(0)
#EXAMDATE	DX_bl	DXCHANGE	AGE	PTGENDER	PTEDUCAT	PTETHCAT	PTRACCAT	PTMARRY	
print(df['PTETHCAT'].head(10))

0    Not Hisp/Latino
1    Not Hisp/Latino
2    Not Hisp/Latino
3    Not Hisp/Latino
4    Not Hisp/Latino
5        Hisp/Latino
6        Hisp/Latino
7        Hisp/Latino
8        Hisp/Latino
9        Hisp/Latino
Name: PTETHCAT, dtype: object


In [235]:
print(df['DXCHANGE'].head(1))
'1=Stable:NL to NL, 2=Stable:MCI to MCI, 3=Stable:AD to AD, ' 
'4=Conv:NL to MCI, 5=Conv:MCI to AD, 6=Conv:NL to AD, ' 
'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

0    1.0
Name: DXCHANGE, dtype: float64


'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

In [236]:

#data = df[['PTGENDER','AGE','PTEDUCAT','APOE4','DXCHANGE','PTETHCAT','PTMARRY','PTEDUCAT','PTRACCAT','EXAMDATE']].copy()
data = df
print(data.shape)

#data.dropna(axis=0,inplace= True)
data['PTGENDER']= data['PTGENDER'].replace(['Male','Female'],[1,0])
data['APOE4']= data['APOE4'].astype(float)
data['AGE']= data['AGE'].astype(float)
data['PTETHCAT']= data['PTETHCAT'].replace(['Hisp/Latino','Not Hisp/Latino','Unknown'],[2,1,0]).astype(int)
data['PTMARRY']= data['PTMARRY'].replace(['Married','Widowed','Divorced','Never married','Unknown'],[0,1,2,3,4]).astype(int)
data['PTRACCAT'] = data['PTRACCAT'].replace(['White','More than one','Black','Asian', 'Am Indian/Alaskan','Hawaiian/Other PI', 'Unknown'],[0,1,2,3,4,5,6]).astype(int)
data['EXAMDATE'] = data['EXAMDATE']
#print(data['EXAMDATE'].unique())

# drop the columns in data they ate objet type
data = data.select_dtypes(exclude=['object'])
print(data.shape)
print(np.where(np.isnan(data)))
#data = np.nan_to_num(data)
print(np.where(np.isnan(data)))

print(data.shape)


(12741, 1907)
(12741, 84)
(array([    0,     0,     0, ..., 12740, 12740, 12740], dtype=int64), array([13, 14, 24, ..., 55, 74, 78], dtype=int64))
(array([    0,     0,     0, ..., 12740, 12740, 12740], dtype=int64), array([13, 14, 24, ..., 55, 74, 78], dtype=int64))
(12741, 84)


In [237]:
# Algorithme des k plus proches voisins
model = KNeighborsClassifier()
y = data['DXCHANGE']
X = data.drop('DXCHANGE', axis=1)
y = np.nan_to_num(y)
X= np.nan_to_num(X)


In [238]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.7055533751962323
0.5810121616320125


In [239]:
param_grid = {'n_neighbors': np.arange(1, 25),'metric': ['euclidean', 'manhattan']}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

C:\Users\antoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [240]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)
model = grid.best_estimator_
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

{'metric': 'manhattan', 'n_neighbors': 1}
0.6400125905971918
KNeighborsClassifier(metric='manhattan', n_neighbors=1)
1.0
0.6673205178501374


In [241]:
confusion_matrix(y_test, model.predict(X_test))


array([[513,  78, 123,  38,   1,   9,   0,   3,   0],
       [ 87, 418,  27,   8,   8,   0,   0,   6,   1],
       [129,  23, 569,  24,   6,  33,   0,   8,   1],
       [ 57,  12,  35, 192,   0,  23,   0,   0,   0],
       [  6,   8,   8,   1,   1,   0,   0,   0,   0],
       [  9,   4,  36,  22,   0,   5,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   1,   7,   0,   2,   0,   0,   3,   0],
       [  0,   0,   0,   1,   0,   1,   0,   0,   0]], dtype=int64)

In [246]:
N, train_score,val_score = learning_curve(model,X_train,y_train,train_sizes = np.linspace(0.2,1.0,5),cv=5)

C:\Users\antoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [253]:
print(N)
plt.subplot(N,train_score.mean(axis=1), label='train')
plt.subplot(N,val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()

[1630 3261 4891 6522 8153]


AttributeError: module 'matplotlib' has no attribute 'subplot'

In [ ]:
def sick(model,PTGENDER=1,AGE=10,PTEDUCAT=4,APOE4=0):
    x= np.array([PTGENDER,AGE,PTEDUCAT,APOE4]).reshape(1,4)
    print(model.predict(x))
    print(model.predict_proba(x))